## `rocky`

In [1]:
import os
os.chdir(r'C:\Users\aweaver\rocky\rocky-app\src\py')
from rockycore import rocky
from triangle import Triangle

import pandas as pd
import numpy as np

In [2]:
# creat a new rocky object
r = rocky()

# inspect object
print(r)

rocky(id='rocky0', mod=(), f=(), plot=(), t=(), rockylog=None)


In [3]:
# rename rocky object
r.rename('andys cool rocky object')
print(r)

rocky(id='andys cool rocky object', mod=(), f=(), plot=(), t=(), rockylog=None)


In [4]:
# add triangle
r.rename('rocky1')
r.SampleTri('mack_1994')
print(r)

rocky(id='rocky1', mod=(), f=(), plot=(), t="rpt_loss", rockylog=None)


In [5]:
# access triangle object
r.rpt_loss

development_period       1        2        3        4        5        6  \
accident_period                                                           
2001-01-01          5012.0   8269.0  10907.0  11805.0  13539.0  16181.0   
2002-01-01           106.0   4285.0   5396.0  10666.0  13782.0  15599.0   
2003-01-01          3410.0   8992.0  13873.0  16141.0  18735.0  22214.0   
2004-01-01          5655.0  11555.0  15766.0  21266.0  23425.0  26083.0   
2005-01-01          1092.0   9565.0  15836.0  22169.0  25955.0  26180.0   
2006-01-01          1513.0   6445.0  11702.0  12935.0  15852.0      NaN   
2007-01-01           557.0   4020.0  10946.0  12314.0      NaN      NaN   
2008-01-01          1351.0   6947.0  13112.0      NaN      NaN      NaN   
2009-01-01          3133.0   5395.0      NaN      NaN      NaN      NaN   
2010-01-01          2063.0      NaN      NaN      NaN      NaN      NaN   

development_period        7        8        9       10  
accident_period                           

In [6]:
# access dataframe representation of triangle object
r.rpt_loss.tri

development_period,1,2,3,4,5,6,7,8,9,10
accident_period,,,,,,,,,,
2001-01-01,5012.0,8269.0,10907.0,11805.0,13539.0,16181.0,18009.0,18608.0,18662.0,18834.0
2002-01-01,106.0,4285.0,5396.0,10666.0,13782.0,15599.0,15496.0,16169.0,16704.0,NaN
2003-01-01,3410.0,8992.0,13873.0,16141.0,18735.0,22214.0,22863.0,23466.0,NaN,NaN
2004-01-01,5655.0,11555.0,15766.0,21266.0,23425.0,26083.0,27067.0,NaN,NaN,NaN
2005-01-01,1092.0,9565.0,15836.0,22169.0,25955.0,26180.0,NaN,NaN,NaN,NaN
2006-01-01,1513.0,6445.0,11702.0,12935.0,15852.0,NaN,NaN,NaN,NaN,NaN
2007-01-01,557.0,4020.0,10946.0,12314.0,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-01,1351.0,6947.0,13112.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-01,3133.0,5395.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# access linear design matrix:
r.rpt_loss.X_base.head(10)

,is_observed,accident_period_2002,accident_period_2003,accident_period_2004,accident_period_2005,accident_period_2006,accident_period_2007,accident_period_2008,accident_period_2009,accident_period_2010,development_period_0002,development_period_0003,development_period_0004,development_period_0005,development_period_0006,development_period_0007,development_period_0008,development_period_0009,development_period_0010
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
6,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
7,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0
8,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0
9,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0


In [8]:
# quick ATA factor summary
r.rpt_loss.ata_summary()

development_period,1,2,3,4,5,6,7,8,9
2001,1.65,1.319,1.082,1.147,1.195,1.113,1.033,1.003,1.009
2002,40.425,1.259,1.977,1.292,1.132,0.993,1.043,1.033,
2003,2.637,1.543,1.163,1.161,1.186,1.029,1.026,,
2004,2.043,1.364,1.349,1.102,1.113,1.038,,,
2005,8.759,1.656,1.4,1.171,1.009,,,,
2006,4.26,1.816,1.105,1.226,,,,,
2007,7.217,2.723,1.125,,,,,,
2008,5.142,1.887,,,,,,,
2009,1.722,,,,,,,,
Vol Wtd,,,,,,,,,


In [9]:
# Triangle data type actually has a robust chain-ladder implementation that can be accessed

# this isn't important for right now, but it is there and it works well:
r.rpt_loss.ult('vwa', 3, excludes='h')

accident_period
2001-01-01    18834.000000
2002-01-01    16857.953917
2003-01-01    24083.370924
2004-01-01    28703.142163
2005-01-01    28427.303039
2006-01-01    18820.427940
2007-01-01    16918.320155
2008-01-01    22196.784045
2009-01-01    18756.915281
2010-01-01    23280.316841
dtype: float64

In [10]:
r.rpt_loss.X_base.loc[r.rpt_loss.X_base.is_observed.eq(1)].index

Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 20, 21, 22, 23, 24, 25, 26, 27, 30, 31, 32, 33, 34, 35, 36, 40, 41,
       42, 43, 44, 45, 50, 51, 52, 53, 54, 60, 61, 62, 63, 70, 71, 72, 80, 81,
       90],
      dtype='int64')

In [11]:
r.rpt_loss.positive_y

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 30, 31, 32, 33, 34, 35, 36,
       40, 41, 42, 43, 44, 45, 50, 51, 52, 53, 54, 60, 62, 63, 70, 71, 72,
       80, 81, 90], dtype=int64)

In [12]:
r.rpt_loss.X_base

,is_observed,accident_period_2002,accident_period_2003,accident_period_2004,accident_period_2005,accident_period_2006,accident_period_2007,accident_period_2008,accident_period_2009,accident_period_2010,development_period_0002,development_period_0003,development_period_0004,development_period_0005,development_period_0006,development_period_0007,development_period_0008,development_period_0009,development_period_0010
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,1,1,1
96,0,1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,1
97,0,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1
98,0,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1


In [13]:
r.rpt_loss.y_base_train.index.max()

90

In [14]:
r.rpt_loss.y_base.iloc[r.rpt_loss.positive_y]

0     5012.0
1      106.0
2     3410.0
3     5655.0
4     1092.0
5     1513.0
6      557.0
7     1351.0
8     3133.0
9     2063.0
10    3257.0
11    4179.0
12    5582.0
13    5900.0
14    8473.0
15    4932.0
16    3463.0
17    5596.0
18    2262.0
20    2638.0
21    1111.0
22    4881.0
23    4211.0
24    6271.0
25    5257.0
26    6926.0
27    6165.0
30     898.0
31    5270.0
32    2268.0
33    5500.0
34    6333.0
35    1233.0
36    1368.0
40    1734.0
41    3116.0
42    2594.0
43    2159.0
44    3786.0
45    2917.0
50    2642.0
51    1817.0
52    3479.0
53    2658.0
54     225.0
60    1828.0
62     649.0
63     984.0
70     599.0
71     673.0
72     603.0
80      54.0
81     535.0
90     172.0
Name: gl_rpt_loss, dtype: float64

In [26]:
# index where y is positive and observed
is_observed = r.rpt_loss.is_observed[r.rpt_loss.positive_y].index.values
is_observed

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 30, 31, 32, 33, 34, 35, 36,
       40, 41, 42, 43, 44, 45, 50, 51, 52, 53, 54, 60, 62, 63, 70, 71, 72,
       80, 81, 90], dtype=int64)

In [28]:
# full design matrix
X = r.rpt_loss.get_X_base()

# drop rows where y is not positive
X = X.iloc[r.rpt_loss.positive_y]

X.index

Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 20, 21, 22, 23, 24, 25, 26, 27, 30, 31, 32, 33, 34, 35, 36, 40, 41,
       42, 43, 44, 45, 50, 51, 52, 53, 54, 60, 62, 63, 70, 71, 72, 80, 81, 90],
      dtype='int64')

In [23]:
# update X_train and X_forecast
X_train = X.iloc[is_observed]
X_forecast = X.iloc[~is_observed]

IndexError: positional indexers are out-of-bounds

In [ ]:
# update y_train
y_train = r.rpt_loss.y_train[r.rpt_loss.tri.positive_y]

# update acc, dev, cal
acc = r.rpt_loss.acc[r.rpt_loss.tri.positive_y]
dev = r.rpt_loss.dev[r.rpt_loss.tri.positive_y]
cal = r.rpt_loss.cal[r.rpt_loss.tri.positive_y]

# update acc_forecast, dev_forecast, cal_forecast
acc_forecast = r.rpt_loss.acc_forecast[~r.rpt_loss.tri.positive_y]
dev_forecast = r.rpt_loss.dev_forecast[~r.rpt_loss.tri.positive_y]
cal_forecast = r.rpt_loss.cal_forecast[~r.rpt_loss.tri.positive_y]

In [15]:
# now add a GLM model to the triangle:
r.AddModel('tweedie', 'tweedie', 'rpt_loss')

IndexError: positional indexers are out-of-bounds

In [ ]:
r.

SyntaxError: invalid syntax (1224273140.py, line 1)

In [ ]:
# load hyperparameters
# r.tweedie.TuneFitHyperparameters()
r.tweedie.SetHyperparameters(alpha=0.3, power=2.3)
r.tweedie

tweedieGLM(alpha=0.3, power=2.3)

In [ ]:
# fit base model to the data
r.tweedie.Fit()
print(r)

ValueError: Some value(s) of y are out of the valid range of the loss 'HalfTweedieLoss'.

In [ ]:
r.tweedie.PredictTriangle().query('development_period==10').yhat.round(0)

90    615110.0
91    601891.0
92    604584.0
93    607350.0
94    609895.0
95    612318.0
96    614165.0
97    615001.0
98    614817.0
99    614521.0
Name: yhat, dtype: float64

In [ ]:
# chain ladder ultimate
atu = r.paid_loss.atu('vwa', 3).tolist()
atu.reverse()

(r.paid_loss.diag().reset_index(drop=True) * pd.Series(atu)).round(0)

0    3901463.0
1    5433719.0
2    5378826.0
3    5297906.0
4    4907781.0
5    5074888.0
6    5524825.0
7    6324694.0
8    5558166.0
9    4853382.0
dtype: float64

In [ ]:
r.tweedie.plot.obs_pred(log=True, color='cal')

In [ ]:
r.tweedie.plot.residual('acc', log=False)

In [ ]:
r.paid_loss.tri

development_period,1,2,3,4,5,6,7,8,9,10
accident_period,,,,,,,,,,
2001-01-01,357848.0,1124788.0,1735330.0,2218270.0,2745596.0,3319994.0,3466336.0,3606286.0,3833515.0,3901463.0
2002-01-01,352118.0,1236139.0,2170033.0,3353322.0,3799067.0,4120063.0,4647867.0,4914039.0,5339085.0,NaN
2003-01-01,290507.0,1292306.0,2218525.0,3235179.0,3985995.0,4132918.0,4628910.0,4909315.0,NaN,NaN
2004-01-01,310608.0,1418858.0,2195047.0,3757447.0,4029929.0,4381982.0,4588268.0,NaN,NaN,NaN
2005-01-01,443160.0,1136350.0,2128333.0,2897821.0,3402672.0,3873311.0,NaN,NaN,NaN,NaN
2006-01-01,396132.0,1333217.0,2180715.0,2985752.0,3691712.0,NaN,NaN,NaN,NaN,NaN
2007-01-01,440832.0,1288463.0,2419861.0,3483130.0,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-01,359480.0,1421128.0,2864498.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-01,376686.0,1363294.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
nm = r.tweedie.GetX().columns[1:].values
nm

array(['accident_period_2002', 'accident_period_2003',
       'accident_period_2004', 'accident_period_2005',
       'accident_period_2006', 'accident_period_2007',
       'accident_period_2008', 'accident_period_2009',
       'accident_period_2010', 'development_period_0002',
       'development_period_0003', 'development_period_0004',
       'development_period_0005', 'development_period_0006',
       'development_period_0007', 'development_period_0008',
       'development_period_0009', 'development_period_0010'], dtype=object)

In [ ]:
r.tweedie.PlotParameter('acc')

In [ ]:
r.tweedie.Combine('dev', [5,6])

development_period_0005
development_period_0006


In [ ]:
r.tweedie.GetX()

,intercept,accident_period_2002,accident_period_2003,accident_period_2004,accident_period_2005,accident_period_2006,accident_period_2007,accident_period_2008,accident_period_2009,accident_period_2010,development_period_0002,development_period_0003,development_period_0004,development_period_0007,development_period_0008,development_period_0009,development_period_0010,dev_combined
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,1,2
41,1,1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1,2
42,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,2
43,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,2


,accident_period,development_period,triangle
0,2001,1,357848.0
1,2002,1,352118.0
2,2003,1,290507.0
3,2004,1,310608.0
4,2005,1,443160.0
5,2006,1,396132.0
6,2007,1,440832.0
7,2008,1,359480.0
8,2009,1,376686.0
9,2010,1,344014.0


In [ ]:
r.paid_loss.diag()

accident_period
2001-01-01    3901463.0
2002-01-01    5339085.0
2003-01-01    4909315.0
2004-01-01    4588268.0
2005-01-01    3873311.0
2006-01-01    3691712.0
2007-01-01    3483130.0
2008-01-01    2864498.0
2009-01-01    1363294.0
2010-01-01     344014.0
dtype: float64

In [ ]:
d = r.paid_loss.diag().reset_index(drop=True)
a = r.paid_loss.atu('vwa')[::-1].reset_index(drop=True)
u = (d * a).round(0)
u.index = r.paid_loss.tri.index.to_series().dt.year
u.index.name = 'acc'
u

acc
2001    3901463.0
2002    5433719.0
2003    5378826.0
2004    5297906.0
2005    4858200.0
2006    5111171.0
2007    5660771.0
2008    6784799.0
2009    5642266.0
2010    4969825.0
dtype: float64

In [ ]:
r.tweedie.Ultimate().round(0)

acc
2001    6921114.0
2002    5971389.0
2003    5690635.0
2004    6686787.0
2005    5489832.0
2006    6127418.0
2007    7078173.0
2008    5745638.0
2009    6331167.0
2010    5891232.0
Name: Ultimate, dtype: float64

In [ ]:
r.paid_loss.ata_summary()

development_period,1,2,3,4,5,6,7,8,9
2001,3.143,1.543,1.278,1.238,1.209,1.044,1.04,1.063,1.018
2002,3.511,1.755,1.545,1.133,1.084,1.128,1.057,1.086,
2003,4.448,1.717,1.458,1.232,1.037,1.12,1.061,,
2004,4.568,1.547,1.712,1.073,1.087,1.047,,,
2005,2.564,1.873,1.362,1.174,1.138,,,,
2006,3.366,1.636,1.369,1.236,,,,,
2007,2.923,1.878,1.439,,,,,,
2008,3.953,2.016,,,,,,,
2009,3.619,,,,,,,,
Vol Wtd,,,,,,,,,


In [ ]:
r.tweedie.PlotParameter('acc')

In [ ]:
df = r.paid_loss.melt_triangle()
df['yhat'] = r.tweedie.GetY().tolist() + r.tweedie.Predict('forecast').round(0).tolist()
df['y_est'] = r.tweedie.Predict().round(0).tolist()
df

,accident_period,development_period,triangle,yhat,y_est
0,2001,1,357848.0,357848.0,620838.0
1,2002,1,352118.0,352118.0,623341.0
2,2003,1,290507.0,290507.0,626042.0
3,2004,1,310608.0,310608.0,628790.0
4,2005,1,443160.0,443160.0,631317.0
...,...,...,...,...,...
95,2006,10,NaN,604855.0,604855.0
96,2007,10,NaN,606597.0,606597.0
97,2008,10,NaN,607362.0,607362.0
98,2009,10,NaN,607138.0,607138.0
